In [3]:
%load_ext autoreload
%autoreload 2

from simphony.libraries.ideal import Waveguide, Coupler
from simphony.circuit import Circuit
from simphony.simulation import ClassicalSim, Laser, Detector
import numpy as jnp

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
a = jnp.arange(16).reshape(4,4)
b = jnp.arange(16).reshape(4,4)
bt = b.T

In [57]:
b

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])

In [58]:
bt

array([[ 0,  4,  8, 12],
       [ 1,  5,  9, 13],
       [ 2,  6, 10, 14],
       [ 3,  7, 11, 15]])

In [61]:
a[0]

array([0, 1, 2, 3])

In [62]:
b[:,0]

array([ 0,  4,  8, 12])

In [63]:
bt[0]

array([ 0,  4,  8, 12])

In [64]:
np.outer(a[0], bt[0])

array([[ 0,  0,  0,  0],
       [ 0,  4,  8, 12],
       [ 0,  8, 16, 24],
       [ 0, 12, 24, 36]])

In [66]:
np.einsum('ij,ik->ijk',a,bt)

array([[[  0,   0,   0,   0],
        [  0,   4,   8,  12],
        [  0,   8,  16,  24],
        [  0,  12,  24,  36]],

       [[  4,  20,  36,  52],
        [  5,  25,  45,  65],
        [  6,  30,  54,  78],
        [  7,  35,  63,  91]],

       [[ 16,  48,  80, 112],
        [ 18,  54,  90, 126],
        [ 20,  60, 100, 140],
        [ 22,  66, 110, 154]],

       [[ 36,  84, 132, 180],
        [ 39,  91, 143, 195],
        [ 42,  98, 154, 210],
        [ 45, 105, 165, 225]]])

In [67]:
a[:,:,None]@bt[:,None,:]

array([[[  0,   0,   0,   0],
        [  0,   4,   8,  12],
        [  0,   8,  16,  24],
        [  0,  12,  24,  36]],

       [[  4,  20,  36,  52],
        [  5,  25,  45,  65],
        [  6,  30,  54,  78],
        [  7,  35,  63,  91]],

       [[ 16,  48,  80, 112],
        [ 18,  54,  90, 126],
        [ 20,  60, 100, 140],
        [ 22,  66, 110, 154]],

       [[ 36,  84, 132, 180],
        [ 39,  91, 143, 195],
        [ 42,  98, 154, 210],
        [ 45, 105, 165, 225]]])

In [12]:
np.outer(a,b)

array([[ 2,  3,  0,  1],
       [ 4,  6,  0,  2],
       [ 6,  9,  0,  3],
       [ 8, 12,  0,  4]])

In [4]:
a = [0,1,4,5]
b = np.array([1,2,3,3])
a < b

array([ True,  True, False, False])

In [28]:
# Laser and detectors added after circuit creation

ckt = Circuit()
wg = Waveguide(length=1)
coupler = Coupler()
ckt.connect(wg.o(0), coupler.o(1))
ckt.connect(wg.o(1), coupler.o(2))
ckt.expose()

laser = Laser(ckt, port=ckt.o(0))
detector = Detector(ckt, port=ckt.o(1))

# wls = np.linspace(1.53, 1.57, 100)
wls = jnp.array([1.55])
sim = ClassicalSim(ckt=ckt, wl=wls)
result = sim.run()
result.output

array([[-0.92994561-0.34037462j]])

In [25]:
# Laser and dectectors added during circuit creation

ckt = Circuit()
wg = Waveguide(length=1)
coupler = Coupler()
laser = Laser()
detector = Detector()
ckt.connect(wg.o(0), coupler.o(1))
ckt.connect(wg.o(1), coupler.o(2))
ckt.connect(laser, coupler)
ckt.connect(detector, coupler)

freq = np.linspace(1.53, 1.57, 100)
sim = ClassicalSim(ckt=ckt, freq=freq)
result = sim.run()

TypeError: Laser.__init__() missing 2 required positional arguments: 'ckt' and 'port'

In [ ]:
# Laser and dectectors added during simulation creation

ckt = Circuit()
wg = Waveguide(length=1)
coupler = Coupler()
ckt.connect(wg.o(0), coupler.o(1))
ckt.connect(wg.o(1), coupler.o(2))
ckt.expose()

freq = np.linspace(1.53, 1.57, 100)
sim = ClassicalSim(ckt=ckt, freq=freq)
laser = Laser(ckt, port=sim.o(0))
detector = Detector(ckt, port=sim.o(1))
result = sim.run()

In [ ]:
# example quantum simulation
from simphony.simulation import QuantumSim, SqueezedState, CoherentState
from simphony.circuit import Circuit

# circcuit 
ckt = Circuit()
wg1 = Waveguide(length=1)
wg2 = Waveguide(length=3)
coupler1 = Coupler(coupling=0.75, loss=0.1)
ckt.connect()

# define input states
state1 = SqueezedState(0.5, 0.5)
state2 = CoherentState(0.5)

freq = 1.55
sim = QuantumSim(ckt=ckt, freq=freq)
